In [1]:
import sys 
sys.path.append('..')



In [2]:
import os
from src.datasets.mimic_cxr_dataset import MIMICCXRDataModule
from pytorch_lightning import Trainer, seed_everything
import torchvision.transforms as T
# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
seed_everything(42)


Global seed set to 42


42

In [3]:

augmentations = {'train':
    T.Compose(
    [
        T.Resize((224, 224)),
        T.RandomApply([T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.8),
        T.RandomGrayscale(p=0.2),
        T.GaussianBlur(kernel_size=9),
        T.ToTensor(),
    ]),
    'val':
    T.Compose(
    [
        T.Resize((224, 224)),
        T.RandomApply([T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.8),
        T.RandomGrayscale(p=0.2),
        T.GaussianBlur(kernel_size=9),
        T.ToTensor(),
    ])
}


In [4]:
data_path = '/Users/caghankoksal/Desktop/development/Flamingo-playground/physionet.org/files/mimic-cxr/2.0.0/files/'
mimic_datamodule = MIMICCXRDataModule(data_path, transforms=augmentations, only_images=False, batch_size=1,
                                limit_num_samples=None, num_data_workers=0, tokenizer="gpt2")
train_loader = mimic_datamodule.train_dataloader()
val_loader = mimic_datamodule.val_dataloader()

100%|██████████| 1/1 [00:00<00:00,  1.53it/s]


In [5]:
len(mimic_datamodule.train_dataset)

4355

In [6]:
len(train_loader)

4355

In [7]:
for batch in train_loader:
    print("Image shape : ",batch["image"].shape)
    print(batch["text"][0])# First sample in the batch
    print("Input Ids shape : ",batch["input_ids"].shape)
    print("Targets shape : ",batch["targets"].shape)
    break

Image shape :  torch.Size([1, 3, 224, 224])
<|endoftext|> <image> Output: FINAL REPORT AP CHEST 1:28 A.M.  HISTORY:  -year-old woman with pneumonia and C. difficile infection. IMPRESSION:  AP chest compared to  through : There has been very little change in the appearance of the lungs in this patient with severe pulmonary fibrosis over the course of this hospitalization, probably a function of either concurrent pneumonia or acceleration of pulmonary fibrosis.  Small bilateral pleural effusions have appeared intermittently, greater today than yesterday, suggesting some component of cardiac decompensation.  ET tube is in standard placement.  PIC line ends in the region of the superior cavoatrial junction alongside a right internal jugular line.  Nasogastric tube passes below the diaphragm and out of view.  No pneumothorax. <EOC>
Input Ids shape :  torch.Size([1, 1, 256])
Targets shape :  torch.Size([1, 1, 256])


In [8]:
BATCH_SIZE = 32
NUM_EPOCHS = 200
len(mimic_datamodule.train_dataset)//BATCH_SIZE*NUM_EPOCHS

27200

In [9]:
mimic_datamodule.train_dataset.tokenizer.all_special_ids[mimic_datamodule.train_dataset.tokenizer.all_special_tokens.index('<image>')]

50258

In [10]:
mimic_datamodule.train_dataset.tokenizer.all_special_ids

[50256, 50257, 50258, 50259]

In [11]:
from src.models.multimodal.flamingo_module import FlamingoModule
import pytorch_lightning as pl

In [24]:
VOCAB_SIZE_OF_TOKENIZER = mimic_datamodule.train_dataset.tokenizer.vocab_size
VOCAB_SIZE_OF_TOKENIZER

50257

In [21]:
LANGUAGE_MODEL = 'gpt2'
NUM_TOKENS = VOCAB_SIZE_OF_TOKENIZER+3 if LANGUAGE_MODEL=="gpt2" else 31092
FLAMINGO_EMBED_DIM = 768
DEPTH = 12
NUM_HEADS = 8
ATT_HEAD_DIM = 64
CROOS_ATT_EVERY=3
MEDIA_TOKEN_ID = mimic_datamodule.train_dataset.tokenizer.all_special_ids[mimic_datamodule.train_dataset.tokenizer.all_special_tokens.index('<image>')]
PERCEIVER_NUM_LATENTS = 64
PERCEIVER_DEPTH = 2
IMAGE_ENCODER = "clip"
LANGUAGE_MODEL = "gpt2"
PRETRAINED_CLIP_PATH = '/Users/caghankoksal/Desktop/development/PubMedCLIP_ViT32.pth'
PRETRAINED_GPT2_PATH = "/Users/caghankoksal/Desktop/development/TransformerPlay/gpt2-pytorch_model.bin"


In [22]:
model = FlamingoModule(pretrained_clip_path = None,
                        total_steps=100, num_tokens = NUM_TOKENS,
                        dim=FLAMINGO_EMBED_DIM, depth=DEPTH, heads=NUM_HEADS, dim_head=ATT_HEAD_DIM,
                        media_token_id=MEDIA_TOKEN_ID, cross_attn_every=CROOS_ATT_EVERY,
                        perceiver_num_latents = PERCEIVER_NUM_LATENTS, perceiver_depth = PERCEIVER_DEPTH,
                        image_encoder =IMAGE_ENCODER, language_model = LANGUAGE_MODEL,
                        pretrained_gpt2_path=PRETRAINED_GPT2_PATH
                        )



Vit is started from scratch
Vit is initialized
Flamingo is being initialized with  gpt2  as language model
GPT 2 Weights are loading...
Loaded GPT2 weights num_weights loaed :  156


In [23]:
trainer = pl.Trainer(max_epochs=200,deterministic=True,
                     accelerator="cpu", devices=1)
trainer.fit(model, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type          | Params
------------------------------------------------
0 | flamingo_palm | FlamingoModel | 202 M 
------------------------------------------------
161 M     Trainable params
41.0 M    Non-trainable params
202 M     Total params
809.749   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]